## Notebook para probar las funciónes de la API

In [1]:
import pandas as pd
import warnings
import numpy as np
import json
from fastapi import FastAPI, Response
from fastapi.responses import JSONResponse
warnings.filterwarnings('ignore')
import datetime

In [2]:
meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
dias_es = ['lunes', 'martes', 'miercoles', 'jueves','viernes','sabado','domingo',"lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]
dias_en = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday","Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Datasets
data = pd.read_csv('../final_data/combined_data.csv',sep=',')
cast = pd.read_csv('../final_data/final_cast.csv')

movies = data[['title','genres','director']]
movies = movies.dropna(subset=['genres']) # Elimino las peliculas que tienen vacio el campo de 'genres'

data.release_date = data.release_date.apply(lambda x: pd.to_datetime(x).date() if '-' in x else x)
data['release_weekday']= data.release_date.apply(lambda x: x.strftime('%A') if type(x) == datetime.date else np.nan)

In [3]:
data.sample(2)

,belongs_to_collection,budget,genres,movie_id,original_language,overview,popularity,production_companies,production_countries,release_date,...,vote_average,vote_count,production_countries_code,release_year,release_month,release_day,return,director,actors,release_weekday
15775,NaN,0.0,Drama|Foreign,8886,de,After the wild life-style of a famous young Ge...,0.595492,Neue Road Movies|P.O.R. Sicilia|Rectangle Prod...,France|Germany|Italy,2010-09-09,...,5.1,14.0,DE|FR|IT,2010,9,9,0.0,Wim Wenders,Dennis Hopper|Milla Jovovich|Giovanna Mezzogio...,Thursday
23797,NaN,0.0,Comedy,53425,en,A quickly made Chaplin two-reeler to help fill...,0.331941,First National Pictures,United States of America,1974-12-25,...,5.6,15.0,US,1974,12,25,0.0,Charlie Chaplin,Charlie Chaplin|Edna Purviance|Jackie Coogan|B...,Wednesday


`def cantidad_filmaciones_mes(mes:str):`

In [6]:
def cantidad_filmaciones_mes(mes:str):
    '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron ese mes historicamente'''
    mes = mes.lower()
    if mes not in meses:
        return {'mensaje':'Ingresar un mes en idioma Español'}
    else:
        mes_numero = meses.index(mes)
        cantidad_xmes = float(data['movie_id'].loc[data.release_month == mes_numero].count())
        data_json = {'mes':mes.capitalize(),'cantidad_fimaciones_mes':cantidad_xmes}

        return data_json

In [7]:
cantidad_filmaciones_mes('marZO')

{'mes': 'Marzo', 'cantidad_fimaciones_mes': 3027.0}

In [5]:
cantidad_filmaciones_mes('OCtubrE')

{'mes': 'octubre', 'cantidad_fimaciones_mes': 4842.0}

`def cantidad_filmaciones_dia(dia:str):`

In [8]:
def cantidad_filmaciones_dia(dia:str):
    '''Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrebaron ese dia historicamente'''
    dia = dia.lower()
    if dia not in dias_es:
        data_json = {'mensaje':'Se requiere de un dia de la semana en idioma Espaniol (Spanish)'}
 
    else:
        dia_index = dias_es.index(dia)
        cantidad_xdia = data.movie_id.loc[data.release_weekday == dias_en[dia_index]].count()
        data_json = {'dia':dia.capitalize(), 'cantidad_filmaciones_dia':int(cantidad_xdia)}

    return data_json

In [9]:
cantidad_filmaciones_dia('JUEves')

{'dia': 'Jueves', 'cantidad_filmaciones_dia': 7528}

`def score_titulo(titulo:str):`

In [12]:
def score_titulo(titulo:str):
    '''Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score'''
    filmaciones = list(data.title.unique())
    if titulo not in filmaciones:
        data_json = {'titulo':'No se encuentra la pelicula en la base de datos'}
    else:
        movie_index = data.loc[data.title == titulo].index
        movie_year = data.release_year.iloc[movie_index]
        movie_score = data.popularity.iloc[movie_index]

        if pd.isna(movie_year).any():
            movie_year = 'Desconocido'
        else:
            movie_year = int(movie_year.iloc[0])

        if pd.isna(movie_score).any():
            movie_score = 'Desconocido'
        else:
            movie_score = float(movie_score.iloc[0])

        data_json = {'titulo':titulo,
                    'anio':movie_year,
                    'popularidad':movie_score}

    return data_json

In [14]:
score_titulo('The Avengers')

{'titulo': 'The Avengers', 'anio': 1998, 'popularidad': 9.562953}

`def votos_titulo(titulo:str):`

In [15]:
def votos_titulo(titulo:str):
    '''Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. 
    La misma variable deberá de contar con al menos 2000 valoraciones, 
    caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.'''
    filmaciones = list(data.title.unique())
    if titulo not in filmaciones:
        data_json = {'titulo': 'Movie not found'}
    else:
        movie_index = data.loc[data.title == titulo].index
        movie_year = data.release_year.iloc[movie_index].tolist()[0] if pd.notna(data.release_year.iloc[movie_index]).any() else 'Desconocido'
        movie_vote_count = data.vote_count.iloc[movie_index].tolist()[0] if pd.notna(data.vote_count.iloc[movie_index]).any() else 'Desconocido'
        movie_vote_average = data.vote_average.iloc[movie_index].tolist()[0] if pd.notna(data.vote_average.iloc[movie_index]).any() else 'Desconocido'

        if pd.notna(movie_vote_count) and int(movie_vote_count) < 2000:
            movie_vote_count = 'Insuficientes votos'
            movie_vote_average = 'Insuficientes votos'
            mensaje = 'Este titulo tiene menos de 2000 votos'

        data_json = {
            'titulo': titulo,
            'anio': int(movie_year) if isinstance(movie_year, float) else movie_year,
            'votos': int(movie_vote_count) if isinstance(movie_vote_count, float) else movie_vote_count,
            'valoracion_promedio': float(movie_vote_average) if isinstance(movie_vote_average, float) else movie_vote_average,
            'mensaje': mensaje if 'mensaje' in locals() else None}
        
    return data_json

In [17]:
votos_titulo('Minions')

{'titulo': 'Minions',
 'anio': 2008,
 'votos': 4729,
 'valoracion_promedio': 6.4,
 'mensaje': None}

`def get_actor(nombre_actor:str):`

In [18]:
def get_actor(nombre_actor:str):
    '''Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, la cantidad de películas que en las que ha participado y el promedio de retorno'''
    all_actors = list(cast.name.unique())

    if nombre_actor not in all_actors:
        data_json = {'mensaje':'Actor not found'}
    else:
        cantidad_titulos_actor = int(data.movie_id.loc[data.actors.fillna('').str.contains(nombre_actor)].count())
        retorno_actor = float(data['return'].loc[data.actors.fillna('').str.contains(nombre_actor)].sum())
        promedio_x_pelicula = retorno_actor / cantidad_titulos_actor

        data_json = {'actor':nombre_actor,
                'cantidad_filmaciones':cantidad_titulos_actor,
                'retorno_total_del_actor':retorno_actor,
                'promedio_de_retorno_por_pelicula':promedio_x_pelicula}
    
    return data_json

In [19]:
get_actor('Tom Hanks')

{'actor': 'Tom Hanks',
 'cantidad_filmaciones': 71,
 'retorno_total_del_actor': 178.8494960552024,
 'promedio_de_retorno_por_pelicula': 2.5190069866929914}

`def get_director(nombre_director:str):`

In [24]:
def get_director(nombre_director:str):
    ''' Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. 
    Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.'''
    
    if nombre_director not in data.director.unique():
        data_json = {'mensaje':'Director no encontrado en la base de datos'}
    else:
        retorno_total_director = data['return'].loc[data.director == nombre_director].sum()
        movies_indexes = data.title.loc[data.director == nombre_director].index
        movies_data = []
        for index in movies_indexes:
            movie_title = data.title.iloc[index]
            movie_release_year = data.release_year.iloc[index]
            movie_return = data['return'].iloc[index]
            movie_budget = data.budget.iloc[index]
            movie_revenue = data.revenue.iloc[index]

            info = {'titulo':movie_title,
                    'anio_estreno':movie_release_year,
                    'retorno_inversion':movie_return,
                    'presupuesto':movie_budget,
                    'ganancias':movie_revenue}
            movies_data.append(info)
            
        data_json = {'director':nombre_director,
                'retorno_total_director':retorno_total_director,
                'peliculas_dirigidas':movies_data}
    
    return data_json

In [25]:
get_director('James Cameron')

{'director': 'James Cameron',
 'retorno_total_director': 54.23441585285313,
 'peliculas_dirigidas': [{'titulo': 'True Lies',
   'anio_estreno': 1994,
   'retorno_inversion': 3.2946296608695653,
   'presupuesto': 115000000.0,
   'ganancias': 378882411.0},
  {'titulo': 'Terminator 2: Judgment Day',
   'anio_estreno': 1991,
   'retorno_inversion': 5.2,
   'presupuesto': 100000000.0,
   'ganancias': 520000000.0},
  {'titulo': 'The Abyss',
   'anio_estreno': 1981,
   'retorno_inversion': 1.2857156857142855,
   'presupuesto': 70000000.0,
   'ganancias': 90000098.0},
  {'titulo': 'Aliens',
   'anio_estreno': 1987,
   'retorno_inversion': 9.908997567567567,
   'presupuesto': 18500000.0,
   'ganancias': 183316455.0},
  {'titulo': 'The Terminator',
   'anio_estreno': 1989,
   'retorno_inversion': 12.2455,
   'presupuesto': 6400000.0,
   'ganancias': 78371200.0},
  {'titulo': 'Titanic',
   'anio_estreno': 1996,
   'retorno_inversion': 9.22517094,
   'presupuesto': 200000000.0,
   'ganancias': 184

---

#### Seleccion de las columnas para la función de `recomendación`
*No selecciono la columna de overview porque si no mi matriz de similitud se va hacer muy pesada y dificil de procesar*

In [30]:
movies = data[['title','genres','director','popularity']]
movies.sample(2)

,title,genres,director,popularity
41408,Wild Things: Foursome,Crime|Drama|Mystery,Andy Hurst,1.932851
8847,Next of Kin,Comedy|Drama,Atom Egoyan,1.046297


In [31]:
movies.genres = movies.genres.str.replace('|',' ')

In [32]:
movies['joined_data'] = movies[['title','genres','director']].astype(str).apply(' '.join, axis=1)
movies.sample()

,title,genres,director,popularity,joined_data
10944,Mission: Impossible III,Action Adventure Thriller,J.J. Abrams,15.687983,Mission: Impossible III Action Adventure Thril...


Cálculo de similitud de puntuación


In [33]:
from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import linear_kernel
 
# Elejí usar TfidfVectorizer para la vectorizacion de los datos en vez de CountVectorizer. Si bien este ultimo es mejor por cuestiones de memoria, 
# el TfidfVectorizer brinda mayor precision a la hora de buscar una puntuación de similitud en las cadenas de strings

In [34]:
vectorizer = TfidfVectorizer(stop_words='english') # Elimino las palabras mas comunes del ingles

In [35]:
movies.genres.isnull().sum()

2383

Veo cuales son las peliculas que tiene en el campo de género un valor vacio y ordeno el resultado dependiendo la popularidad de la misma

In [36]:
movies.loc[movies.genres.isnull()].sort_values(by='popularity', ascending=False).head(20)

,title,genres,director,popularity,joined_data
20850,The Scapegoat,NaN,Charles Sturridge,7.618217,The Scapegoat nan Charles Sturridge
7889,Fahrenheit 9/11,NaN,Michael Moore,6.839460,Fahrenheit 9/11 nan Michael Moore
30783,My Date with the President's Daughter,NaN,Alex Zamm,5.019029,My Date with the President's Daughter nan Alex...
44772,Notte prima degli esami - Oggi,NaN,Fausto Brizzi,4.345520,Notte prima degli esami - Oggi nan Fausto Brizzi
25866,Viaggi di nozze,NaN,Carlo Verdone,4.323640,Viaggi di nozze nan Carlo Verdone
37395,La Vacanza,NaN,Tinto Brass,4.158529,La Vacanza nan Tinto Brass
43072,I still hide to smoke,NaN,Rayhana Obermeyer,3.589741,I still hide to smoke nan Rayhana Obermeyer
32019,School of Thieves,NaN,Neri Parenti,3.080456,School of Thieves nan Neri Parenti
32024,Fracchia The Human Beast,NaN,Neri Parenti,3.053669,Fracchia The Human Beast nan Neri Parenti
30589,Koyla,NaN,Rakesh Roshan,3.037553,Koyla nan Rakesh Roshan


Como no son peliculas muy conocidas y tampoco son muchas, decido eliminarlas del dataset. Tomo esta decision porque al momento de recurrir a la función de recomendación de película, el modelo de machine learning debe seleccionar entre las peliculas que tienen el mismo género cinematografico para relacionarlas y en los casos de estas peliculas seria irrelevante ya que no tienen defino un género.

In [37]:
movies = movies.dropna(subset='genres')

In [38]:
movies.title.sample().values

array(['Traffic'], dtype=object)

### Desarrollo de la función

In [53]:
def recomendacion(title):
    """Get the recommendation for a given title"""
    # Obtengo los géneros de la pelicula
    if title not in movies.title.unique():
        data_json = {'mensaje':'Pelicula no encontrada'}
    else:
        # Obtengo los géneros de la pelicula
        movie_genres = movies['genres'].loc[movies['title'] == title]
        #movie_genres = movie_genres.values[pd.isna(movie_genres.values) == False].item()
        movie_genres = sorted(movie_genres, key= lambda x: len(x), reverse=True) # Ordenar los géneros en función del len()


        # Separo los generos en una lista
        #genres = [v.split(' ') for v in movie_genres if isinstance(v,str)]

        # Filtro el dataframe con las peliculas que tengan los mismos géneros que la pelicula de interes
        # y con las peliculas que tengan en su titulo el nombre de la pelicula de interes
        similar_movies = movies.loc[(movies.genres.fillna(' ').str.contains(movie_genres[0])) | (movies.title.str.contains(title))].reset_index(drop=True)
        max_len = 3850
        if len(similar_movies) > max_len:
            similar_movies = similar_movies.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)[:max_len]

        # Obtengo el indice en el dataframe filtrado de la pelicula de interes
        movie_index = similar_movies.loc[similar_movies['title'] == title].index
        
        if len(movie_index) == 0:
            similar_movies = similar_movies.append(movies.loc[movies.title == title]).reset_index(drop=True)
            movie_index = similar_movies.loc[similar_movies['title'] == title].index

        # Vectorizo los datos
        data_matrix = vectorizer.fit_transform(similar_movies.joined_data)
        # Genero la matriz de puntuación de similitud entre las peliculas
        similarity_matrix = cosine_similarity(data_matrix,data_matrix)

        sim_movies = list(enumerate(similarity_matrix[movie_index[0]]))
        sim_movies = sorted(sim_movies, key=lambda x: x[1], reverse=True)
        top_similar_movies = [similar_movies.loc[i, 'title'] for i, _ in sim_movies[1:20] if similar_movies.loc[i, 'title'] != title][:5]
        data_json = {'peliculas_similares':top_similar_movies}

    
    return data_json

In [59]:
recomendacion('Kill for Me')

{'peliculas_similares': ['Kill Me Again',
  'A Job to Kill For',
  '3 Days to Kill',
  'Enough',
  'A Time to Kill']}